In [45]:
import numpy as np
import json
import csv
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
import optuna
%matplotlib inline

In [46]:
def load_data(y):
    path = '../../data/std_data/'
    x_train_std = pd.read_pickle(path +'train/{}_x.pkl'.format(str(y))).values
    x_test_std = pd.read_pickle(path +'test/{}_x.pkl'.format(str(y))).values
    y_train = pd.read_pickle(path +'train/{}_y.pkl'.format(str(y))).values
    y_test = pd.read_pickle(path +'test/{}_y.pkl'.format(str(y))).values
    features = pd.read_pickle(path +'train/{}_x.pkl'.format(str(y))).columns
    return x_train_std, x_test_std, y_train, y_test, features

In [47]:
def print_weight_map(weight, features):
    # feature importance
    feature_imp = weight.reshape(features.shape)
    feature_imp_arg = np.argsort(feature_imp)
    f_num = len(feature_imp_arg)
    plt.rcParams["font.family"] = "IPAexGothic"
    plt.figure(figsize=(20,20))
    plt.barh(range(len(feature_imp_arg)), feature_imp[feature_imp_arg], color='b', align='center')
    plt.yticks(range(len(feature_imp_arg)), features[feature_imp_arg])
    plt.show()

In [48]:
def plot_roc_curve(fpr, tpr, auc):
    # ROC曲線をプロット
    plt.plot(fpr, tpr, label='ROC curve (area = %.2f)'%auc)
    plt.legend()
    plt.title('ROC curve')
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.grid(True)
    plt.show()

In [54]:
# optunaで最適なC_valを探す

def train(trial):
    
    th_prob = 0.5
    C_val = trial.suggest_uniform('C_val', 0.00001, 0.0001)
    
    print(" C_val:{}".format(C_val))
    
    cm_all = np.zeros((2, 2))
    
    # 予測した確率全体を格納
    probs_all = np.array([])
    y_true_all = np.array([])
    
    for y in range(1978, 2020):
        
        x_train_std, x_test_std, y_train, y_test, features = load_data(y)
        
        # 学習
        lr = LogisticRegression(class_weight="balanced", solver="liblinear",  penalty="l2", C=C_val) # ロジスティック回帰モデルのインスタンスを作成

        lr.fit(x_train_std, y_train) # ロジスティック回帰モデルの重みを学習
        
        # 予測(確率)
        probs = lr.predict_proba(x_test_std)
        probs_all = np.hstack((probs_all, probs[:,1]))
        y_true_all = np.hstack((y_true_all, y_test))

        # 混同行列
        y_pred = np.where((probs[:,1] ==  max(probs[:,1])) | (probs[:,1] > th_prob), 1, 0) #確率→0/1
        cm = confusion_matrix(y_true=y_test, y_pred=y_pred)
        cm_all += cm
        
       # 各特徴量の重み係数
        weight = lr.coef_
        
        
#         print_weight_map(weight, features)

    auc = roc_auc_score(y_true_all, probs_all)
    fpr, tpr, thresholds = roc_curve(y_true_all, probs_all)
    
#     print_weight_map(weight, features)
#     plot_roc_curve(fpr, tpr, auc)
    
#     print("len: {0} , {1}".format(len(y_true_all), len(probs_all) ))
#     print("confusion_matrix: ")
#     print(cm_all)
#     print("AUC: ")
#     print(auc)
#     print()
    return -auc

In [50]:
# from sklearn.utils.testing import ignore_warnings
# from sklearn.exceptions import ConvergenceWarning
# @ignore_warnings(category=ConvergenceWarning)
def main():
    np.set_printoptions(precision=3, suppress=True)
    study = optuna.create_study()
    study.optimize(train, n_trials=100)
    print(study.best_trial)

In [51]:
if __name__ == "__main__":
    main()

 C_val:0.3525798674357871


[I 2019-08-23 01:42:12,853] Finished trial#0 resulted in value: -0.6293322062552832. Current best value is -0.6293322062552832 with parameters: {'C_val': 0.3525798674357871, 'l1_ratio': 0.7063360278300304}.


 C_val:0.0019865883386039826


[I 2019-08-23 01:42:13,166] Finished trial#1 resulted in value: -0.5011975204282897. Current best value is -0.6293322062552832 with parameters: {'C_val': 0.3525798674357871, 'l1_ratio': 0.7063360278300304}.


 C_val:0.24285830690831434


...

 C_val:0.10443771618656783


[I 2019-08-23 01:43:32,560] Finished trial#98 resulted in value: -0.6501831501831501. Current best value is -0.7383770076077769 with parameters: {'C_val': 0.0006166771719882391, 'l1_ratio': 0.003510689537417372}.


 C_val:0.028100717035856107


[I 2019-08-23 01:43:33,108] Finished trial#99 resulted in value: -0.7206255283178359. Current best value is -0.7383770076077769 with parameters: {'C_val': 0.0006166771719882391, 'l1_ratio': 0.003510689537417372}.


FrozenTrial(number=82, state=<TrialState.COMPLETE: 1>, value=-0.7383770076077769, datetime_start=datetime.datetime(2019, 8, 23, 1, 43, 20, 332483), datetime_complete=datetime.datetime(2019, 8, 23, 1, 43, 20, 809710), params={'C_val': 0.0006166771719882391, 'l1_ratio': 0.003510689537417372}, distributions={'C_val': UniformDistribution(low=1e-05, high=1), 'l1_ratio': UniformDistribution(low=0, high=1)}, user_attrs={}, system_attrs={'_number': 82}, intermediate_values={}, params_in_internal_repr={'C_val': 0.0006166771719882391, 'l1_ratio': 0.003510689537417372}, trial_id=82)


# 結果
C = 0.00003

In [53]:
# solver="liblinear",  penalty="l2",
#best value is -0.7483798253029023 with parameters: {'C_val': 2.9301597434339928e-05}.

# l1 and l2
# best value is -0.7383770076077769 with parameters: {'C_val': 0.0006166771719882391, 'l1_ratio': 0.003510689537417372}.